In [2]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import roc_curve, auc, roc_auc_score
import imblearn
import warnings
import plotly
import matplotlib.pyplot as plt
import seaborn as sns
import PIL
import optuna

%matplotlib inline

warnings.filterwarnings("ignore")

import catboost as cb
import lightgbm as lgb
import xgboost as xgb
import ngboost as ngb

pd.set_option('display.max_columns',100)

In [3]:
train_raw = pd.read_csv('C:/Users/ASUS/Desktop/isic-2024-challenge/train-metadata.csv')
test_raw = pd.read_csv('C:/Users/ASUS/Desktop/isic-2024-challenge/test-metadata.csv')
train_raw.shape, test_raw.shape

((401059, 55), (3, 44))

In [23]:
train_raw.head(5)

,isic_id,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,tbp_lv_L,tbp_lv_Lext,tbp_lv_areaMM2,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_deltaA,tbp_lv_deltaB,tbp_lv_deltaL,tbp_lv_deltaLB,tbp_lv_deltaLBnorm,tbp_lv_eccentricity,tbp_lv_location,tbp_lv_location_simple,tbp_lv_minorAxisMM,tbp_lv_nevi_confidence,tbp_lv_norm_border,tbp_lv_norm_color,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence,target
0,ISIC_0015670,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,16.261975,26.922447,23.954773,33.684638,28.953117,53.058545,55.828924,54.367448,62.025701,3.152561,27.476170,0.00000,3.982447,2.967674,-7.658253,8.360566,5.784302,0.901302,Right Leg - Upper,Right Leg,1.543016,2.628592e-03,7.091360,0.000000,9.307003,0.00000,2.036195,2.637780,0.590476,85,-182.703552,613.493652,-42.427948,Memorial Sloan Kettering Cancer Center,CC-BY,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282,0
1,ISIC_0015845,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,25.364740,26.331000,24.549290,41.219030,35.299260,39.702910,44.064040,48.861520,55.362360,0.919497,12.235290,0.00000,6.347830,1.781713,-6.500838,6.839008,4.987244,0.639885,Head & Neck,Head & Neck,0.821918,1.334303e-07,2.116402,0.000000,3.354148,0.00000,0.853227,3.912844,0.285714,55,-0.078308,1575.687000,57.174500,Memorial Sloan Kettering Cancer Center,CC-BY,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455,0
2,ISIC_0015864,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,17.128170,37.970460,33.485410,44.174920,37.611800,59.265850,62.909730,53.961180,61.670520,3.265153,24.184620,0.00000,5.447655,4.485044,-7.709336,9.092376,6.290359,0.932147,Torso Back Top Third,Torso Back,1.194905,2.959177e-04,4.798335,0.000000,8.886309,0.00000,1.743651,1.950777,0.361905,105,123.649700,1472.010000,232.908900,Memorial Sloan Kettering Cancer Center,CC-BY,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040,0
3,ISIC_0015902,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,12.164757,21.448144,21.121356,25.746200,24.374023,56.414429,60.060388,18.649518,23.314841,6.079940,14.889242,0.51452,2.077572,0.326788,-4.665323,4.783413,6.400196,0.654458,Torso Front Top Half,Torso Front,2.481328,2.198945e+01,1.975874,1.771705,9.514499,0.66469,1.258541,1.573733,0.209581,130,-141.024780,1442.185791,58.359802,ACEMID MIA,CC-0,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998,0
4,ISIC_0024200,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,20.057470,26.464900,25.710460,36.217980,32.608740,46.946070,52.041180,46.276310,54.855740,2.101708,19.902560,0.00000,4.668053,0.754434,-8.579431,9.148495,6.531302,0.946448,Torso Front Top Half,Torso Front,0.929916,1.378832e-03,3.658854,0.000000,6.467562,0.00000,2.085409,2.480509,0.313433,20,-72.315640,1488.720000,21.428960,Memorial Sloan Kettering Cancer Center,CC-BY,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510,0


In [25]:
test_raw.head()

,isic_id,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,tbp_lv_L,tbp_lv_Lext,tbp_lv_areaMM2,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_deltaA,tbp_lv_deltaB,tbp_lv_deltaL,tbp_lv_deltaLB,tbp_lv_deltaLBnorm,tbp_lv_eccentricity,tbp_lv_location,tbp_lv_location_simple,tbp_lv_minorAxisMM,tbp_lv_nevi_confidence,tbp_lv_norm_border,tbp_lv_norm_color,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license,target
0,ISIC_0015657,IP_6074337,45.0,male,posterior torso,2.70,TBP tile: close-up,3D: XP,22.80433,20.007270,28.38412,27.043640,36.410100,33.640000,51.220960,53.505430,24.97985,31.114600,3.846876,22.907010,0.461149,2.797056,1.340481,-6.134747,6.436557,6.843057,0.664465,Torso Back Top Third,Torso Back,2.187644,1.698104e-02,5.435366,1.143374,9.387248,0.304827,1.281532,2.299935,0.479339,20,-155.06510,1511.222000,113.980100,Memorial Sloan Kettering Cancer Center,CC-BY,0
1,ISIC_0015729,IP_1664139,35.0,female,lower extremity,2.52,TBP tile: close-up,3D: XP,16.64867,9.657964,31.31752,27.524318,35.467806,29.169579,62.004494,70.664619,59.90409,68.141071,2.120473,18.957821,0.000000,6.990705,3.793202,-8.236981,9.151127,6.083388,0.926698,Left Leg - Upper,Left Leg,1.032666,2.107364e-01,4.322201,0.000000,6.340311,0.000000,1.271940,2.011223,0.426230,25,-112.36924,629.535889,-15.019287,"Frazer Institute, The University of Queensland...",CC-BY,0
2,ISIC_0015740,IP_7142616,65.0,male,posterior torso,3.16,TBP tile: close-up,3D: XP,24.25384,19.937380,30.46368,28.384240,38.939500,34.686660,51.474730,54.915410,35.81945,41.358640,3.396510,19.464400,0.251236,4.316465,2.079433,-5.539191,6.041092,5.446997,0.894776,Torso Back Top Third,Torso Back,1.520786,8.052259e-13,3.968912,0.721739,8.130868,0.230742,1.080308,2.705857,0.366071,110,-84.29282,1303.978000,-28.576050,FNQH Cairns,CC-BY,0


<b>The Columns in both the <code>train-metadata.csv</code> and <code>test-metadata.csv</code></b>

<table>
<thead>
<tr>
<th>field name</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>isic_id</code></td>
<td>Unique case identifier.</td>
</tr>
<tr>
<td><code>patient_id</code>&nbsp;</td>
<td>Unique patient identifier.</td>
</tr>
<tr>
<td><code>age_approx</code></td>
<td>Approximate age of patient at time of imaging.</td>
</tr>
<tr>
<td><code>sex</code></td>
<td>Sex of the person.</td>
</tr>
<tr>
<td><code>anatom_site_general</code></td>
<td>Location of the lesion on the patient's body.</td>
</tr>
<tr>
<td><code>clin_size_long_diam_mm</code></td>
<td>Maximum diameter of the lesion (mm).+</td>
</tr>
<tr>
<td><code>image_type</code></td>
<td>Structured field of the ISIC Archive for image type.</td>
</tr>
<tr>
<td><code>tbp_tile_type</code></td>
<td>Lighting modality of the 3D TBP source image.</td>
</tr>
<tr>
<td><code>tbp_lv_A</code></td>
<td>A inside lesion.+</td>
</tr>
<tr>
<td><code>tbp_lv_Aex</code></td>
<td>A outside lesion.+</td>
</tr>
<tr>
<td><code>tbp_lv_B</code></td>
<td>B inside lesion.+</td>
</tr>
<tr>
<td><code>tbp_lv_Bext</code></td>
<td>B outside lesion.+</td>
</tr>
<tr>
<td><code>tbp_lv_C</code></td>
<td>Chroma inside lesion.+</td>
</tr>
<tr>
<td><code>tbp_lv_Cext</code></td>
<td>Chroma outside lesion.+</td>
</tr>
<tr>
<td><code>tbp_lv_H</code></td>
<td>Hue inside the lesion; calculated as the angle of A* and B* in L<em>A</em>B* color space. Typical values range from 25 (red) to 75 (brown).+</td>
</tr>
<tr>
<td><code>tbp_lv_Hext</code></td>
<td>Hue outside lesion.+</td>
</tr>
<tr>
<td><code>tbp_lv_L</code></td>
<td>L inside lesion.+</td>
</tr>
<tr>
<td><code>tbp_lv_Lext</code></td>
<td>L outside lesion.+</td>
</tr>
<tr>
<td><code>tbp_lv_areaMM2</code></td>
<td>Area of lesion (mm^2).+</td>
</tr>
<tr>
<td><code>tbp_lv_area_perim_ratio</code></td>
<td>Border jaggedness, the ratio between lesions perimeter and area. Circular lesions will have low values; irregular shaped lesions will have higher values. Values range 0-10.+</td>
</tr>
<tr>
<td><code>tbp_lv_color_std_mean</code></td>
<td>Color irregularity, calculated as the variance of colors within the lesion's boundary.</td>
</tr>
<tr>
<td><code>tbp_lv_deltaA</code></td>
<td>Average A contrast (inside vs. outside lesion).+</td>
</tr>
<tr>
<td><code>tbp_lv_deltaB</code></td>
<td>Average B contrast (inside vs. outside lesion).+</td>
</tr>
<tr>
<td><code>tbp_lv_deltaL</code></td>
<td>Average L contrast (inside vs. outside lesion).+</td>
</tr>
<tr>
<td><code>tbp_lv_deltaLBnorm</code></td>
<td>Contrast between the lesion and its immediate surrounding skin. Low contrast lesions tend to be faintly visible such as freckles; high contrast lesions tend to be those with darker pigment. Calculated as the average delta L<em>B</em> of the lesion relative to its immediate background in L<em>A</em>B* color space. Typical values range from 5.5 to 25.+</td>
</tr>
<tr>
<td><code>tbp_lv_eccentricity</code></td>
<td>Eccentricity.+</td>
</tr>
<tr>
<td><code>tbp_lv_location</code></td>
<td>Classification of anatomical location, divides arms &amp; legs to upper &amp; lower; torso into thirds.+</td>
</tr>
<tr>
<td><code>tbp_lv_location_simple</code></td>
<td>Classification of anatomical location, simple.+</td>
</tr>
<tr>
<td><code>tbp_lv_minorAxisMM</code></td>
<td>Smallest lesion diameter (mm).+</td>
</tr>
<tr>
<td><code>tbp_lv_nevi_confidence</code></td>
<td>Nevus confidence score (0-100 scale) is a convolutional neural network classifier estimated probability that the lesion is a nevus. The neural network was trained on approximately 57,000 lesions that were classified and labeled by a dermatologist.+,++</td>
</tr>
<tr>
<td><code>tbp_lv_norm_border</code></td>
<td>Border irregularity (0-10 scale); the normalized average of border jaggedness and asymmetry.+</td>
</tr>
<tr>
<td><code>tbp_lv_norm_color</code></td>
<td>Color variation (0-10 scale); the normalized average of color asymmetry and color irregularity.+</td>
</tr>
<tr>
<td><code>tbp_lv_perimeterMM</code></td>
<td>Perimeter of lesion (mm).+</td>
</tr>
<tr>
<td><code>tbp_lv_radial_color_std_max</code></td>
<td>Color asymmetry, a measure of asymmetry of the spatial distribution of color within the lesion. This score is calculated by looking at the average standard deviation in L<em>A</em>B* color space within concentric rings originating from the lesion center. Values range 0-10.+</td>
</tr>
<tr>
<td><code>tbp_lv_stdL</code></td>
<td>Standard deviation of L inside lesion.+</td>
</tr>
<tr>
<td><code>tbp_lv_Lext</code></td>
<td>Standard deviation of L outside lesion.+</td>
</tr>
<tr>
<td><code>tbp_lv_symm_2axis</code></td>
<td>Border asymmetry; a measure of asymmetry of the lesion's contour about an axis perpendicular to the lesion's most symmetric axis. Lesions with two axes of symmetry will therefore have low scores (more symmetric), while lesions with only one or zero axes of symmetry will have higher scores (less symmetric). This score is calculated by comparing opposite halves of the lesion contour over many degrees of rotation. The angle where the halves are most similar identifies the principal axis of symmetry, while the second axis of symmetry is perpendicular to the principal axis. Border asymmetry is reported as the asymmetry value about this second axis. Values range 0-10.+</td>
</tr>
<tr>
<td><code>tbp_lv_symm_2axis_angle</code></td>
<td>Lesion border asymmetry angle.+</td>
</tr>
<tr>
<td><code>tbp_lv_x</code></td>
<td>X-coordinate of the lesion on 3D TBP.+</td>
</tr>
<tr>
<td><code>tbp_lv_y</code></td>
<td>Y-coordinate of the lesion on 3D TBP.+</td>
</tr>
<tr>
<td><code>tbp_lv_z</code></td>
<td>Z-coordinate of the lesion on 3D TBP.+</td>
</tr>
<tr>
<td><code>attribution</code></td>
<td>Image attribution, synonymous with image source.</td>
</tr>
<tr>
<td><code>copyright_license</code></td>
<td>Copyright license.</td>
</tr>
</tbody>
</table>

In [14]:
train_raw.describe().T

,count,mean,std,min,25%,50%,75%,max
target,401059.0,0.000980,0.031288,0.000000e+00,0.000000,0.000000,0.000000,1.000000
age_approx,398261.0,58.012986,13.596165,5.000000e+00,50.000000,60.000000,70.000000,85.000000
clin_size_long_diam_mm,401059.0,3.930827,1.743068,1.000000e+00,2.840000,3.370000,4.380000,28.400000
tbp_lv_A,401059.0,19.974007,3.999489,-2.487115e+00,17.330821,19.801910,22.304628,48.189610
tbp_lv_Aext,401059.0,14.919247,3.529384,-9.080269e+00,12.469740,14.713930,17.137175,37.021680
tbp_lv_B,401059.0,28.281706,5.278676,-7.309895e-01,24.704372,28.171570,31.637429,54.306900
tbp_lv_Bext,401059.0,26.913015,4.482994,9.237066e+00,23.848125,26.701704,29.679913,48.372700
tbp_lv_C,401059.0,34.786341,5.708469,3.054228e+00,31.003148,34.822580,38.430298,58.765170
tbp_lv_Cext,401059.0,30.921279,4.829345,1.184652e+01,27.658285,30.804893,33.963868,54.305290
tbp_lv_H,401059.0,54.653689,5.520849,-1.574164e+00,51.566273,55.035632,58.298184,105.875784


In [16]:
train_raw.columns, test_raw.columns

(Index(['isic_id', 'target', 'patient_id', 'age_approx', 'sex',
        'anatom_site_general', 'clin_size_long_diam_mm', 'image_type',
        'tbp_tile_type', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext',
        'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L',
        'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio',
        'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB',
        'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm',
        'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_location_simple',
        'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border',
        'tbp_lv_norm_color', 'tbp_lv_perimeterMM',
        'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt',
        'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y',
        'tbp_lv_z', 'attribution', 'copyright_license', 'lesion_id',
        'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5',
        'mel_mi

In [4]:
target = train_raw['target']
train_raw.drop(['target'], axis=1, inplace=True)
train_raw['target'] = target
test_raw['target'] = 0

common_columns = train_raw.columns.intersection(test_raw.columns)
print(common_columns)

train_raw = train_raw[common_columns]

df_train_test = pd.concat([train_raw, test_raw], axis=0)
df_train_test.drop(columns=['image_type','attribution','copyright_license'],axis=1,inplace=True,errors='ignore')
df_train_test

Index(['isic_id', 'patient_id', 'age_approx', 'sex', 'anatom_site_general',
       'clin_size_long_diam_mm', 'image_type', 'tbp_tile_type', 'tbp_lv_A',
       'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext',
       'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2',
       'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean', 'tbp_lv_deltaA',
       'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLB',
       'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity', 'tbp_lv_location',
       'tbp_lv_location_simple', 'tbp_lv_minorAxisMM',
       'tbp_lv_nevi_confidence', 'tbp_lv_norm_border', 'tbp_lv_norm_color',
       'tbp_lv_perimeterMM', 'tbp_lv_radial_color_std_max', 'tbp_lv_stdL',
       'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle',
       'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z', 'attribution', 'copyright_license',
       'target'],
      dtype='object')


,isic_id,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,tbp_lv_L,tbp_lv_Lext,tbp_lv_areaMM2,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_deltaA,tbp_lv_deltaB,tbp_lv_deltaL,tbp_lv_deltaLB,tbp_lv_deltaLBnorm,tbp_lv_eccentricity,tbp_lv_location,tbp_lv_location_simple,tbp_lv_minorAxisMM,tbp_lv_nevi_confidence,tbp_lv_norm_border,tbp_lv_norm_color,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,target
0,ISIC_0015670,IP_1235828,60.0,male,lower extremity,3.04,3D: white,20.244422,16.261975,26.922447,23.954773,33.684638,28.953117,53.058545,55.828924,54.367448,62.025701,3.152561,27.476170,0.000000,3.982447,2.967674,-7.658253,8.360566,5.784302,0.901302,Right Leg - Upper,Right Leg,1.543016,2.628592e-03,7.091360,0.000000,9.307003,0.000000,2.036195,2.637780,0.590476,85,-182.703552,613.493652,-42.427948,0
1,ISIC_0015845,IP_8170065,60.0,male,head/neck,1.10,3D: white,31.712570,25.364740,26.331000,24.549290,41.219030,35.299260,39.702910,44.064040,48.861520,55.362360,0.919497,12.235290,0.000000,6.347830,1.781713,-6.500838,6.839008,4.987244,0.639885,Head & Neck,Head & Neck,0.821918,1.334303e-07,2.116402,0.000000,3.354148,0.000000,0.853227,3.912844,0.285714,55,-0.078308,1575.687000,57.174500,0
2,ISIC_0015864,IP_6724798,60.0,male,posterior torso,3.40,3D: XP,22.575830,17.128170,37.970460,33.485410,44.174920,37.611800,59.265850,62.909730,53.961180,61.670520,3.265153,24.184620,0.000000,5.447655,4.485044,-7.709336,9.092376,6.290359,0.932147,Torso Back Top Third,Torso Back,1.194905,2.959177e-04,4.798335,0.000000,8.886309,0.000000,1.743651,1.950777,0.361905,105,123.649700,1472.010000,232.908900,0
3,ISIC_0015902,IP_4111386,65.0,male,anterior torso,3.22,3D: XP,14.242329,12.164757,21.448144,21.121356,25.746200,24.374023,56.414429,60.060388,18.649518,23.314841,6.079940,14.889242,0.514520,2.077572,0.326788,-4.665323,4.783413,6.400196,0.654458,Torso Front Top Half,Torso Front,2.481328,2.198945e+01,1.975874,1.771705,9.514499,0.664690,1.258541,1.573733,0.209581,130,-141.024780,1442.185791,58.359802,0
4,ISIC_0024200,IP_8313778,55.0,male,anterior torso,2.73,3D: white,24.725520,20.057470,26.464900,25.710460,36.217980,32.608740,46.946070,52.041180,46.276310,54.855740,2.101708,19.902560,0.000000,4.668053,0.754434,-8.579431,9.148495,6.531302,0.946448,Torso Front Top Half,Torso Front,0.929916,1.378832e-03,3.658854,0.000000,6.467562,0.000000,2.085409,2.480509,0.313433,20,-72.315640,1488.720000,21.428960,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401057,ISIC_9999964,IP_5231513,30.0,female,anterior torso,2.80,3D: XP,22.288570,9.564721,28.431200,27.012250,36.126360,28.655640,51.905420,70.501540,49.328490,60.874160,6.023644,13.787630,1.317875,12.723850,1.418951,-11.545660,11.751160,8.515440,0.492929,Torso Front Bottom Half,Torso Front,2.547945,9.931933e+01,1.421536,3.583966,9.113276,1.078204,3.680175,1.957157,0.161850,140,-9.861557,877.527000,-76.982120,0
401058,ISIC_9999967,IP_6426047,50.0,male,lower extremity,3.30,3D: XP,16.792900,11.434730,21.457220,21.214910,27.247270,24.100320,51.952470,61.675560,26.342490,38.377250,6.905611,14.856770,0.978867,5.358174,0.242311,-12.034770,12.157000,12.033930,0.667850,Left Leg - Lower,Left Leg,2.583116,9.365141e+01,1.883554,3.100943,10.128920,1.088073,3.068755,3.199104,0.197917,45,-258.872300,148.606200,107.710600,0
0,ISIC_0015657,IP_6074337,45.0,male,posterior torso,2.70,3D: XP,22.804330,20.007270,28.384120,27.043640,36.410100,33.640000,51.220960,53.505430,24.979850,31.114600,3.846876,22.907010,0.461149,2.797056,1.340481,-6.134747,6.436557,6.843057,0.664465,Torso Back Top Third,Torso Back,2.187644,1.698104e-02,5.435366,1.143374,9.387248,0.304827,1.281532,2.299935,0.479339,20,-155.065100,15

In [5]:
Encoder_cols = ['sex','anatom_site_general',
                'tbp_tile_type',
                'tbp_lv_location',
                'tbp_lv_location_simple']

for cols in df_train_test.columns:
    if cols in Encoder_cols:
        print(df_train_test[cols].unique(), df_train_test[cols].isnull().sum())
    else:
        print(cols, df_train_test[cols].isnull().sum())


isic_id 0
patient_id 0
age_approx 2798
['male' 'female' nan] 11517
['lower extremity' 'head/neck' 'posterior torso' 'anterior torso'
 'upper extremity' nan] 5756
clin_size_long_diam_mm 0
['3D: white' '3D: XP'] 0
tbp_lv_A 0
tbp_lv_Aext 0
tbp_lv_B 0
tbp_lv_Bext 0
tbp_lv_C 0
tbp_lv_Cext 0
tbp_lv_H 0
tbp_lv_Hext 0
tbp_lv_L 0
tbp_lv_Lext 0
tbp_lv_areaMM2 0
tbp_lv_area_perim_ratio 0
tbp_lv_color_std_mean 0
tbp_lv_deltaA 0
tbp_lv_deltaB 0
tbp_lv_deltaL 0
tbp_lv_deltaLB 0
tbp_lv_deltaLBnorm 0
tbp_lv_eccentricity 0
['Right Leg - Upper' 'Head & Neck' 'Torso Back Top Third'
 'Torso Front Top Half' 'Right Arm - Upper' 'Left Leg - Upper'
 'Torso Front Bottom Half' 'Left Arm - Upper' 'Right Leg'
 'Torso Back Middle Third' 'Right Arm - Lower' 'Right Leg - Lower'
 'Left Leg - Lower' 'Left Arm - Lower' 'Unknown' 'Left Leg'
 'Torso Back Bottom Third' 'Left Arm' 'Right Arm' 'Torso Front'
 'Torso Back'] 0
['Right Leg' 'Head & Neck' 'Torso Back' 'Torso Front' 'Right Arm'
 'Left Leg' 'Left Arm' 'Unknown'] 0

In [6]:
from sklearn.preprocessing import OrdinalEncoder

def StrFeaturesEncode(df:pd.DataFrame, cols:list) -> pd.DataFrame:
    FeatureEncoder = OrdinalEncoder(categories='auto', 
                                    dtype=int, 
                                    handle_unknown='use_encoded_value', 
                                    unknown_value=-2, 
                                    encoded_missing_value=-1)
    cols_Encoded = FeatureEncoder.fit_transform(df[Encoder_cols])
    for x, col in enumerate(cols):
        df[col] =  cols_Encoded[:,x]
    return df

df_train_test_Encoded = StrFeaturesEncode(df_train_test, Encoder_cols)
df_train_test_Encoded 

,isic_id,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,tbp_lv_L,tbp_lv_Lext,tbp_lv_areaMM2,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_deltaA,tbp_lv_deltaB,tbp_lv_deltaL,tbp_lv_deltaLB,tbp_lv_deltaLBnorm,tbp_lv_eccentricity,tbp_lv_location,tbp_lv_location_simple,tbp_lv_minorAxisMM,tbp_lv_nevi_confidence,tbp_lv_norm_border,tbp_lv_norm_color,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,target
0,ISIC_0015670,IP_1235828,60.0,1,2,3.04,1,20.244422,16.261975,26.922447,23.954773,33.684638,28.953117,53.058545,55.828924,54.367448,62.025701,3.152561,27.476170,0.000000,3.982447,2.967674,-7.658253,8.360566,5.784302,0.901302,12,4,1.543016,2.628592e-03,7.091360,0.000000,9.307003,0.000000,2.036195,2.637780,0.590476,85,-182.703552,613.493652,-42.427948,0
1,ISIC_0015845,IP_8170065,60.0,1,1,1.10,1,31.712570,25.364740,26.331000,24.549290,41.219030,35.299260,39.702910,44.064040,48.861520,55.362360,0.919497,12.235290,0.000000,6.347830,1.781713,-6.500838,6.839008,4.987244,0.639885,0,0,0.821918,1.334303e-07,2.116402,0.000000,3.354148,0.000000,0.853227,3.912844,0.285714,55,-0.078308,1575.687000,57.174500,0
2,ISIC_0015864,IP_6724798,60.0,1,3,3.40,0,22.575830,17.128170,37.970460,33.485410,44.174920,37.611800,59.265850,62.909730,53.961180,61.670520,3.265153,24.184620,0.000000,5.447655,4.485044,-7.709336,9.092376,6.290359,0.932147,16,5,1.194905,2.959177e-04,4.798335,0.000000,8.886309,0.000000,1.743651,1.950777,0.361905,105,123.649700,1472.010000,232.908900,0
3,ISIC_0015902,IP_4111386,65.0,1,0,3.22,0,14.242329,12.164757,21.448144,21.121356,25.746200,24.374023,56.414429,60.060388,18.649518,23.314841,6.079940,14.889242,0.514520,2.077572,0.326788,-4.665323,4.783413,6.400196,0.654458,19,6,2.481328,2.198945e+01,1.975874,1.771705,9.514499,0.664690,1.258541,1.573733,0.209581,130,-141.024780,1442.185791,58.359802,0
4,ISIC_0024200,IP_8313778,55.0,1,0,2.73,1,24.725520,20.057470,26.464900,25.710460,36.217980,32.608740,46.946070,52.041180,46.276310,54.855740,2.101708,19.902560,0.000000,4.668053,0.754434,-8.579431,9.148495,6.531302,0.946448,19,6,0.929916,1.378832e-03,3.658854,0.000000,6.467562,0.000000,2.085409,2.480509,0.313433,20,-72.315640,1488.720000,21.428960,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401057,ISIC_9999964,IP_5231513,30.0,0,0,2.80,0,22.288570,9.564721,28.431200,27.012250,36.126360,28.655640,51.905420,70.501540,49.328490,60.874160,6.023644,13.787630,1.317875,12.723850,1.418951,-11.545660,11.751160,8.515440,0.492929,18,6,2.547945,9.931933e+01,1.421536,3.583966,9.113276,1.078204,3.680175,1.957157,0.161850,140,-9.861557,877.527000,-76.982120,0
401058,ISIC_9999967,IP_6426047,50.0,1,2,3.30,0,16.792900,11.434730,21.457220,21.214910,27.247270,24.100320,51.952470,61.675560,26.342490,38.377250,6.905611,14.856770,0.978867,5.358174,0.242311,-12.034770,12.157000,12.033930,0.667850,5,2,2.583116,9.365141e+01,1.883554,3.100943,10.128920,1.088073,3.068755,3.199104,0.197917,45,-258.872300,148.606200,107.710600,0
0,ISIC_0015657,IP_6074337,45.0,1,3,2.70,0,22.804330,20.007270,28.384120,27.043640,36.410100,33.640000,51.220960,53.505430,24.979850,31.114600,3.846876,22.907010,0.461149,2.797056,1.340481,-6.134747,6.436557,6.843057,0.664465,16,5,2.187644,1.698104e-02,5.435366,1.143374,9.387248,0.304827,1.281532,2.299935,0.479339,20,-155.065100,1511.222000,113.980100,0
1,ISIC_0015729,IP_1664139,35.0,0,2,2.52,0,16.648670,9.657964,31.317520,27.524318,35.467806,29.169579,62.004494,70.664619,59.904090,68.141071,2.120473,18.957821,0.000000,6.990705,3.793202,-8.236981,9.151127,6.083388,0.926698,6,2,1.032666,2.107364e-01,4.322201,0.000000,6.340311,0.000000,1.271940,2.011223,0.426230,25,-112.369240,629.535889,-15.019287,0


In [7]:
train_df = df_train_test_Encoded[:len(train_raw)]
test_df = df_train_test_Encoded[len(train_raw):]

train_df['img_path'] = "/kaggle/input/isic-2024-challenge" 
test_df['img_path'] += f''

In [8]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

def comp_score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80):
    v_gt = abs(np.asarray(solution.values)-1)
    v_pred = np.array([1.0 - x for x in submission.values])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc

def custom_lgbm_metric(y_true, y_hat):
    # TODO: Refactor with the above.
    min_tpr = 0.80
    v_gt = abs(y_true-1)
    v_pred = np.array([1.0 - x for x in y_hat])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return "pauc80", partial_auc, True

In [9]:
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier


N_SPLITS = 5
skf = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True)
for idx, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df["target"], groups=train_df["patient_id"])):
    train_df.loc[val_idx, "fold"] = idx

In [10]:
train_cols = ['age_approx', 'sex', 'anatom_site_general',
       'clin_size_long_diam_mm', 'tbp_tile_type', 'tbp_lv_A', 'tbp_lv_Aext',
       'tbp_lv_B', 'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H',
       'tbp_lv_Hext', 'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2',
       'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean', 'tbp_lv_deltaA',
       'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLB',
       'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity', 'tbp_lv_location',
       'tbp_lv_location_simple', 'tbp_lv_minorAxisMM',
       'tbp_lv_nevi_confidence', 'tbp_lv_norm_border', 'tbp_lv_norm_color',
       'tbp_lv_perimeterMM', 'tbp_lv_radial_color_std_max', 'tbp_lv_stdL',
       'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle',
       'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z']

In [11]:
train_df["target"].value_counts()

target
0    400666
1       393
Name: count, dtype: int64

# LightGBM

In [130]:
import optuna
import optuna_dashboard

OPTIMIZE_OPTUNA = True

def objective_lgbm(trial):
    param_grid = {
        "objective": trial.suggest_categorical("objective", ["binary"]),
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    scores = []
    for fold in range(N_SPLITS):
        train_fold = train_df[train_df['fold'] != f].reset_index(drop=True)
        valid_fold = train_df[train_df['fold'] == f].reset_index(drop=True)
        dtrain = lgb.Dataset(train_fold[train_cols], label=train_fold["target"])
        gbm = lgb.train(param_grid, dtrain)
        preds = gbm.predict(valid_fold[train_cols])
        score = comp_score(valid_fold[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")
        scores.append(score)
    return np.mean(scores)


if OPTIMIZE_OPTUNA:
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lgbm, n_trials=20)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))


[I 2024-07-31 15:40:55,027] A new study created in memory with name: no-name-c09fe356-e55e-467c-8d55-c26cc679897c
[I 2024-07-31 15:41:03,174] Trial 0 finished with value: 0.12364372555946737 and parameters: {'objective': 'binary', 'lambda_l1': 0.3092063742003385, 'lambda_l2': 8.728757122705432e-06, 'num_leaves': 148, 'feature_fraction': 0.9873380300217464, 'bagging_fraction': 0.5211386472265052, 'bagging_freq': 7, 'min_child_samples': 95}. Best is trial 0 with value: 0.12364372555946737.
[I 2024-07-31 15:41:12,626] Trial 1 finished with value: 0.1346267679413305 and parameters: {'objective': 'binary', 'lambda_l1': 0.07608852924193055, 'lambda_l2': 0.1628813550812703, 'num_leaves': 96, 'feature_fraction': 0.8981230705102525, 'bagging_fraction': 0.6607874589241951, 'bagging_freq': 5, 'min_child_samples': 57}. Best is trial 1 with value: 0.1346267679413305.
[I 2024-07-31 15:41:24,752] Trial 2 finished with value: 0.08317242681545187 and parameters: {'objective': 'binary', 'lambda_l1': 0.0

Number of finished trials: 20
Best trial:
  Value: 0.14054414828170106
  Params: 
    objective: binary
    lambda_l1: 6.971443591087062e-06
    lambda_l2: 0.1298238749167169
    num_leaves: 83
    feature_fraction: 0.7040622142457744
    bagging_fraction: 0.6717261975577635
    bagging_freq: 3
    min_child_samples: 85


In [137]:
lgb_parameters = {
    "objective": "binary",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "n_estimators": 200,
    'learning_rate': 0.05,    
    'lambda_l1': 0.000006971443591087062, 
    'lambda_l2': 0.1298238749167169, 
    'num_leaves': 136, 
    'feature_fraction': 0.7040622142457744, 
    'bagging_fraction': 0.6717261975577635, 
    'bagging_freq': 5,
    'min_child_samples': 85,
    "device": "cpu"
}
lgb_scores = []
lgb_models = []
oof_df = pd.DataFrame()
for f in range(N_SPLITS):
    train_fold = train_df[train_df['fold'] != f].reset_index(drop=True)
    valid_fold = train_df[train_df['fold'] == f].reset_index(drop=True)
    
    LGB_model = VotingClassifier([(f"lgb_{i}", lgb.LGBMClassifier(random_state=i, **lgb_parameters)) for i in range(3)], voting="soft")
    LGB_model.fit(train_fold[train_cols], train_fold['target'])
    preds = LGB_model.predict_proba(valid_fold[train_cols])[:, 1]
    
    score = comp_score(valid_fold[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")
    print(f"fold: {f+1} - Partial AUC Score: {score:.5f}")
    lgb_models.append(LGB_model)
    oof_single = valid_fold[["isic_id", "target"]].copy()
    oof_single["pred"] = preds
    oof_df = pd.concat([oof_df, oof_single])

fold: 1 - Partial AUC Score: 0.15043
fold: 2 - Partial AUC Score: 0.17172
fold: 3 - Partial AUC Score: 0.17082
fold: 4 - Partial AUC Score: 0.15390
fold: 5 - Partial AUC Score: 0.14788


In [139]:
lgbm_score = comp_score(oof_df["target"], oof_df["pred"], "")
print(f"LGBM Score: {lgbm_score:.5f}")

LGBM Score: 0.15721


# img

# Submission

In [144]:
lgb_preds = np.mean([model.predict_proba(test_df[train_cols])[:, 1] for model in lgb_models], 0)
df_sub = pd.read_csv("isic-2024-challenge/sample_submission.csv")
df_sub["target"] = lgb_preds
df_sub

,isic_id,target
0,ISIC_0015657,2.491654e-06
1,ISIC_0015729,9.805662e-07
2,ISIC_0015740,8.968716e-06
